In [305]:
import pennylane as qml
import numpy as np
import tensorflow as tf

In [306]:
dev = qml.device('default.qubit', wires=5)

In [307]:
def real(angles, **kwargs):
    qml.Hadamard(wires=0)
    qml.RY(np.pi / 4, wires=1)
    qml.RX(np.pi / 4, wires=0)
    qml.CNOT(wires=[0, 1])
    qml.Rot(*angles, wires=0)
    qml.CNOT(wires=[0, 1])

In [308]:
def generator(w, wires, **kwargs):
    qml.Hadamard(wires=wires[0])
    qml.RX(w[0], wires=wires[0])
    qml.RX(w[1], wires=wires[1])
    qml.RY(w[2], wires=wires[0])
    qml.RY(w[3], wires=wires[1])
    qml.RZ(w[4], wires=wires[0])
    qml.RZ(w[5], wires=wires[1])
    qml.CNOT(wires=[wires[0], wires[1]])
    qml.RX(w[6], wires=wires[0])
    qml.RY(w[7], wires=wires[0])
    qml.RZ(w[8], wires=wires[0])


In [309]:
def utheta(theta, wires, **kwargs):
    qml.Hadamard(wires=wires[0])
    qml.RX(theta[0], wires=wires[0])
    qml.RY(theta[2], wires=wires[0])
    qml.RZ(theta[4], wires=wires[0])

In [310]:
def utheta_dagger(theta, wires, **kwargs):
    qml.RZ(-theta[4], wires=wires[0])
    qml.RY(-theta[2], wires=wires[0])
    qml.RX(-theta[0], wires=wires[0])
    qml.Hadamard(wires=wires[0])

In [311]:
def diag(wires):
    qml.Hadamard(wires=4)
    qml.CZ(wires=[4, wires])

In [312]:
@qml.qnode(dev, interface="tf")
def qpca_train(angles, theta_weights, **kwargs):
    real(angles)
    utheta(theta_weights, [0, 1])
    z0 = qml.expval(qml.PauliZ(0))
    z1 = qml.expval(qml.PauliZ(1))
    return z0, z1

In [313]:
@qml.qnode(dev, interface="tf")
def qpca(angles, theta_weights, **kwargs):
    real(angles)
    utheta(theta_weights, [0, 1])
    z0 = qml.expval(qml.PauliX(0))
    z1 = qml.expval(qml.PauliX(1))
    return z0, z1

In [314]:
def pcaloss(angles, theta_weights, **kwargs):
    z0, z1 = qpca_train(angles, theta_weights)
    return 0.5 * ((z0 + 1) + 2 * (z1 + 1))

In [315]:
angles = [np.pi / 6, np.pi / 2, np.pi / 7]

np.random.seed(0)
eps = 1e-2
init_theta_weights = np.array([0] + [0] * 8) + \
                   np.random.normal(scale=eps, size=(9,))
theta_weights = tf.Variable(init_theta_weights)

opt = tf.keras.optimizers.SGD(0.4)

In [316]:
cost = lambda: pcaloss(angles, theta_weights)
for step in range(200):
    opt.minimize(cost, theta_weights)
    if step % 5 == 0:
        cost_val = cost().numpy()
        print("Step {}: cost = {}".format(step, cost_val))

Step 0: cost = 1.3343848928140136
Step 5: cost = 1.1669167553300674
Step 10: cost = 1.124091565221662
Step 15: cost = 1.1130562258685588
Step 20: cost = 1.108921407716011
Step 25: cost = 1.1069228758807754
Step 30: cost = 1.1058774166505283
Step 35: cost = 1.1053246881146088
Step 40: cost = 1.1050337388797096
Step 45: cost = 1.10488134690854
Step 50: cost = 1.104801783672349
Step 55: cost = 1.104760316863267
Step 60: cost = 1.1047387243076106
Step 65: cost = 1.104727485456612
Step 70: cost = 1.1047216368448922
Step 75: cost = 1.1047185935528765
Step 80: cost = 1.1047170100595192
Step 85: cost = 1.1047161861478896
Step 90: cost = 1.1047157574599034
Step 95: cost = 1.1047155344108808
Step 100: cost = 1.1047154183572903
Step 105: cost = 1.104715357974038
Step 110: cost = 1.1047153265563472
Step 115: cost = 1.1047153102095775
Step 120: cost = 1.104715301704278
Step 125: cost = 1.1047152972789314
Step 130: cost = 1.1047152949764036
Step 135: cost = 1.1047152937783884
Step 140: cost = 1.1047

In [317]:
@qml.qnode(dev, interface="tf")
def get_real_prob(angles, theta_weights, **kwargs):
    real(angles)
    utheta(theta_weights, [0, 1])
    return qml.probs(wires=[0])


In [318]:
prob_real = get_real_prob(angles, theta_weights)
purity_real = np.linalg.norm(prob_real) ** 2
print(prob_real)
print(purity_real)

tf.Tensor([0.10471529 0.89528471], shape=(2,), dtype=float64)
0.8124999999995197


In [319]:
@qml.qnode(dev, interface="tf")
def get_gen_prob(gen_weights, theta_weights, **kwargs):
    generator(gen_weights, [2, 3])
    utheta(theta_weights, [2, 3])
    return qml.probs(wires=[2])

In [320]:
@qml.qnode(dev, interface="tf")
def get_gen_purity(gen_weights, theta_weights, **kwargs):
    generator(gen_weights, [0, 1])
    utheta(theta_weights, [0, 1])
    generator(gen_weights, [2, 3])
    utheta(theta_weights, [2, 3])
    qml.Hadamard(wires=4)
    qml.CSWAP(wires=[4, 0, 2])
    qml.Hadamard(wires=4)
    return qml.expval(qml.PauliZ(4))


In [321]:
init_gen_weights = np.array([np.pi] + [0] * 8) + \
                   np.random.normal(scale=eps, size=(9,))
gen_weights = tf.Variable(init_gen_weights)

In [322]:
def gen_loss(gen_weights, theta_weights):
    global prob_real
    prob_gen = get_gen_prob(gen_weights, theta_weights)
    #purity_gen = get_gen_purity(gen_weights, theta_weights)
    return - tf.reduce_mean(tf.reduce_sum(prob_real*tf.math.log(prob_gen)))# - (purity_real * tf.math.log(purity_gen) + (1 - purity_real) * tf.math.log(1 - purity_gen))
    #return tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = prob_gen, labels = prob_real))

In [323]:
cost = lambda: gen_loss(gen_weights, theta_weights)
for step in range(100):
    opt.minimize(cost, gen_weights)
    if step % 5 == 0:
        cost_val = cost().numpy()
        print("Step {}: cost = {}".format(step, cost_val))

Step 0: cost = 0.5418757806332488
Step 5: cost = 0.3456920095912377
Step 10: cost = 0.3355782625431037
Step 15: cost = 0.33532687896085056
Step 20: cost = 0.3353217947530322
Step 25: cost = 0.3353216953925264
Step 30: cost = 0.3353216934602643
Step 35: cost = 0.33532169342271395
Step 40: cost = 0.3353216934219846
Step 45: cost = 0.33532169342197
Step 50: cost = 0.3353216934219705
Step 55: cost = 0.33532169342197027
Step 60: cost = 0.33532169342197
Step 65: cost = 0.33532169342197055
Step 70: cost = 0.33532169342197016
Step 75: cost = 0.3353216934219697
Step 80: cost = 0.33532169342196994
Step 85: cost = 0.3353216934219701
Step 90: cost = 0.3353216934219696
Step 95: cost = 0.3353216934219695


In [324]:
@qml.qnode(dev, interface="tf")
def show_real_diag(angles, theta_weights, **kwargs):
    real(angles)
    utheta(theta_weights, [0, 1])
    return qml.density_matrix([0])

@qml.qnode(dev, interface="tf")
def show_gen_diag(gen_weights, theta_weights, **kwargs):
    generator(gen_weights, [2, 3])
    utheta(theta_weights, [2, 3])
    return qml.density_matrix([2])
    
print(show_real_diag(angles, theta_weights))
print(show_gen_diag(gen_weights, theta_weights))

tf.Tensor(
[[1.04715292e-01+0.00000000e+00j 9.14546025e-09+4.89576263e-07j]
 [9.14546025e-09-4.89576263e-07j 8.95284708e-01+0.00000000e+00j]], shape=(2, 2), dtype=complex128)
tf.Tensor(
[[ 0.10471529+0.j         -0.12751678-0.03828425j]
 [-0.12751678+0.03828425j  0.89528471+0.j        ]], shape=(2, 2), dtype=complex128)


In [325]:
@qml.qnode(dev, interface="tf")
def show_real(angles):
    real(angles)
    return qml.density_matrix([0])

@qml.qnode(dev, interface="tf")
def show_gen(gen_weights, theta_weights):
    generator(gen_weights, [2, 3])
    utheta(theta_weights, [2, 3])
    diag(2)
    utheta_dagger(theta_weights, [2, 3])
    return qml.density_matrix([2])
    
print(show_real(angles))
print(show_gen(gen_weights, theta_weights))

tf.Tensor(
[[ 0.19381378+0.j         -0.10847093-0.22524222j]
 [-0.10847093+0.22524222j  0.80618622+0.j        ]], shape=(2, 2), dtype=complex128)
tf.Tensor(
[[ 0.19381378+0.j       -0.10847138-0.225242j]
 [-0.10847138+0.225242j  0.80618622+0.j      ]], shape=(2, 2), dtype=complex128)


In [326]:
def new_generator(weights, **kwargs):
    gen_weights, theta_weights = weights[0], weights[1]
    generator(gen_weights, [0, 1])
    utheta(theta_weights, [0, 1])
    diag(0)
    utheta_dagger(theta_weights, [0, 1])

In [327]:
obs = [qml.PauliX(0), qml.PauliY(0), qml.PauliZ(0)]

bloch_vector_real = qml.map(real, obs, dev, interface="tf")
bloch_vector_generator = qml.map(new_generator, obs, dev, interface="tf")

print("Real Bloch vector: {}".format(bloch_vector_real(angles)))
print("Generator Bloch vector: {}".format(bloch_vector_generator([gen_weights, theta_weights])))

Real Bloch vector: [-0.21694187  0.45048443 -0.61237244]
Generator Bloch vector: [-0.21694275  0.45048401 -0.61237244]
